In [32]:
with open("secret_claude_key.txt", 'r') as file:
    secret_key = file.read().strip()

In [7]:
import pandas as pd

df = pd.read_csv("link_verification_with_text_cleaned.csv")
df.head()

,Link,Element Exists,Plain Text
0,https://thenevadaindependent.com/article/despi...,True,"By Graham Lee Brewer, Associated Press OKLAHOM..."
1,https://thenevadaindependent.com/article/analy...,True,Editor's Note: We recommend using Google Chrom...
2,https://thenevadaindependent.com/article/trump...,True,Elias Benjelloun’s parents were issued their d...
3,https://thenevadaindependent.com/article/whats...,True,"Ace Acosta, who, on top of his job as a food m..."
4,https://thenevadaindependent.com/article/nevad...,True,Nevada Secretary of State Cisco Aguilar on Thu...


In [33]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=secret_key,
)

In [38]:
# Define a function to analyze each article
def analyze_article(news_article):
    """
    Sends a news article to the Anthropic API for political analysis and categorization.
    """
    try:
        # Define the message to send to the Anthropic assistant
        message = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1000,
            temperature=0,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"You are a skilled political analyst tasked with analyzing a news article to determine which political group would be most satisfied with its content and what category the news belongs to. Your analysis should be impartial and based solely on the provided content.\n\nHere is the news article you need to analyze:\n\n<{news_article}>\n{{news_article}}\n</news_article>\n\nPlease follow these steps in your analysis:\n\n1. Read the news article carefully.\n2. Analyze the content for political bias, tone, and implications that might appeal to a specific political group.\n3. Determine which of the following groups would be most satisfied with the article's content:\n   - Republican\n   - Democrat\n   - Neutral (if the article appears unbiased or equally satisfying/unsatisfying to both major parties)\n4. Categorize the news article into one of the following topics:\n   - Traits (referring to traits of candidates)\n   - Economy\n   - Domestic policy\n   - Defense\n   - Foreign policy\n\n5. Provide your analysis inside <political_breakdown> tags, considering the following:\n   a. List and quote key statements that might indicate political leaning, categorizing them as Republican-leaning, Democrat-leaning, or Neutral.\n   b. Analyze the tone and language used in the article.\n   c. Consider the presentation of facts and any potential omissions.\n   d. Identify key topics or themes in the article that relate to the given categories.\n   e. Summarize your findings before making a final decision.\n\n6. After your analysis, provide your final output in the following format:\n\n<political_analysis>\n[Your detailed analysis of the article's political implications]\n</political_analysis>\n\n<response>\n[Single word: Republican, Democrat, or Neutral]\n</response>\n\n<category>\n[Single word: Traits, Economy, DomesticPolicy, Defense, or ForeignPolicy]\n</category>\n\nRemember, your <response> and <category> must each be a single word to facilitate processing of large numbers of articles."
                        }
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "<political_breakdown>"
                        }
                    ]
                }
            ]
        )
        # Extract the response content
        return message.content
    except Exception as e:
        # Handle any errors gracefully
        return f"Error: {e}"

In [40]:
# test one article
result = analyze_article(df["Plain Text"].iloc[0])

In [42]:
result[0].text

'\nKey statements analysis:\n\nDemocrat-leaning statements:\n- "systemic barriers to voting on tribal lands contribute to substantial disparities in Native American turnout"\n- "voters were denied the ability to vote using their tribal IDs in several places"\n- Discussion of voting access barriers and their negative impact\n- Emphasis on need for Native American Voting Rights Act\n\nRepublican-leaning statements:\n- None significant\n\nNeutral statements:\n- Statistical data and research findings\n- Description of Nevada\'s voting systems\n- Discussion of data collection challenges\n\nTone Analysis:\nThe article maintains a scholarly tone, focusing on research findings and factual reporting. However, the emphasis on voting barriers and disparities aligns more closely with traditional Democratic Party concerns about voting access and minority rights.\n\nLanguage Analysis:\nUses terms like "systemic barriers," "disparities," and "inequities" which are more commonly associated with Democr

In [44]:
# let's experiment with batches
# take the first 10 articles
articles = df["Plain Text"].iloc[0:10]
articles

0    By Graham Lee Brewer, Associated Press OKLAHOM...
1    Editor's Note: We recommend using Google Chrom...
2    Elias Benjelloun’s parents were issued their d...
3    Ace Acosta, who, on top of his job as a food m...
4    Nevada Secretary of State Cisco Aguilar on Thu...
5    Culture war issues centered in the Clark Count...
6    By Nicholas Riccardi, Associated Press The 202...
7    The wind was gusting outside the South Valleys...
8    A presidential candidate with a unique ability...
9    Officials in Nevada’s two most populous counti...
Name: Plain Text, dtype: object

In [45]:
def make_request(news_article, id):
    return {
        "custom_id": str(id),
        "params": {
            "model": "claude-3-5-sonnet-20241022",
            "max_tokens": 1000,
            "temperature": 0,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"You are a skilled political analyst tasked with analyzing a news article to determine which political group would be most satisfied with its content and what category the news belongs to. Your analysis should be impartial and based solely on the provided content.\n\nHere is the news article you need to analyze:\n\n<news_article>\n{news_article}\n</news_article>\n\nPlease follow these steps in your analysis:\n\n1. Read the news article carefully.\n2. Analyze the content for political bias, tone, and implications that might appeal to a specific political group.\n3. Determine which of the following groups would be most satisfied with the article's content:\n   - Republican\n   - Democrat\n   - Neutral (if the article appears unbiased or equally satisfying/unsatisfying to both major parties)\n4. Categorize the news article into one of the following topics:\n   - Traits (referring to traits of candidates)\n   - Economy\n   - Domestic policy\n   - Defense\n   - Foreign policy\n\n5. Provide your analysis inside <political_breakdown> tags, considering the following:\n   a. List and quote key statements that might indicate political leaning, categorizing them as Republican-leaning, Democrat-leaning, or Neutral.\n   b. Analyze the tone and language used in the article.\n   c. Consider the presentation of facts and any potential omissions.\n   d. Identify key topics or themes in the article that relate to the given categories.\n   e. Summarize your findings before making a final decision.\n\n6. After your analysis, provide your final output in the following format:\n\n<political_analysis>\n[Your detailed analysis of the article's political implications]\n</political_analysis>\n\n<response>\n[Single word: Republican, Democrat, or Neutral]\n</response>\n\n<category>\n[Single word: Traits, Economy, DomesticPolicy, Defense, or ForeignPolicy]\n</category>\n\nRemember, your <response> and <category> must each be a single word to facilitate processing of large numbers of articles."
                        }
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "<political_breakdown>"
                        }
                    ]
                }
            ]
        }
    }

message_batch = client.beta.messages.batches.create(
    requests=[
        make_request(article, id) for id, article in enumerate(articles)
    ]
)
print(message_batch)

BetaMessageBatch(id='msgbatch_01UXdfJWf4tEVgSA3ZKUS33a', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2024, 12, 10, 0, 17, 10, 630839, tzinfo=datetime.timezone.utc), ended_at=None, expires_at=datetime.datetime(2024, 12, 11, 0, 17, 10, 630839, tzinfo=datetime.timezone.utc), processing_status='in_progress', request_counts=BetaMessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=10, succeeded=0), results_url=None, type='message_batch')


In [46]:
# works!
# let's do it for all
articles = df["Plain Text"]

message_batch = client.beta.messages.batches.create(
    requests=[
        make_request(article, id) for id, article in enumerate(articles)
    ]
)